### Data Ingesion - Research code 

In [1]:
import os

In [3]:
%pwd

'd:\\workspace-mlops\\MLOps-Project-02-End-to-End-Data-Science-Project-with-Deployment\\research'

In [4]:
os.chdir('../')
%pwd

'd:\\workspace-mlops\\MLOps-Project-02-End-to-End-Data-Science-Project-with-Deployment'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

    

In [6]:
from src.datasience.constants import *
from src.datasience.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(self, 
                 config_filepath=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH,
                 schema_filepath=SCHEMA_FILE_PATH
                 ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL = config.source_URL,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )

        return data_ingestion_config

In [12]:
from urllib import request
from src.datasience import logger
import zipfile

In [13]:
## Component -- Data Ingestion 

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
    
    # Download the zip file 
    def dowload_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists")

    # Extract zip file 
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [16]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config = data_ingestion_config)
    data_ingestion.dowload_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-11-24 18:56:09,384: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-11-24 18:56:09,388: INFO: common: yaml file: params.yaml loaded successfully]
[2024-11-24 18:56:09,393: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-11-24 18:56:09,396: INFO: common: created directory at: artifacts]
[2024-11-24 18:56:09,398: INFO: common: created directory at: artifacts/data_ingestion]
[2024-11-24 18:56:11,389: INFO: 1901970137: artifacts/data_ingestion/data.zip download! with following info: 
Date: Sun, 24 Nov 2024 13:56:08 GMT
Content-Type: application/octet-stream
Content-Length: 23329
Connection: close
cache-control: max-age=60, public, must-revalidate, stale-while-revalidate=60, stale-if-error=300, s-maxage=60
content-disposition: attachment
content-security-policy: base-uri 'self'; child-src https://www.google.com/recaptcha/ https://www.recaptcha.net/ https://www.googletagmanager.com/ns.html https://*.zuora.com/apps/PublicHostedPageLite.do https://g